# Mapping wildfire burn scars using HLS data

📥 [Download notebook](https://github.com/Beldine-Moturi/AMLD-Africa-2026/raw/main/Example_notebooks/GeospatialStudio-Walkthrough-BurnScars.ipynb) and try it out!

Imagine that you work in disaster response and need a rapid way to map the extent areas burned by wildfires.  You need to do this in an automated, scalable manner.  We can achieve this using an AI model which ingests satellite data (in this instance the NASA Harmonized Landsat Sentinel2 dataset) and outputs a map of burned area.  We could potentially then integrate the burned area extent with details of infrastructure or assets to quantify impact.

<!-- ![alt text](assets/burn-scars-examples.png) -->
<img src="../images/burn-scars-examples.png" alt="drawing" width="600"/>

In this walkthrough we will assume that a model doesn't exist yet and we want to train a new model.  We will then show how to drive the model to map impact.

We will walk through the following steps as part of this walkthrough:
1. Upload and onboarding of data
2. Configuring and submitting a tuning task
3. Monitoring model training
4. Testing and validation of the outputs

## Pre-requisites
This walkthrough assumes you have the data downloaded locally, it can be downloaded here: https://s3.us-east.cloud-object-storage.appdomain.cloud/geospatial-studio-example-data/burn-scar-training-data.zip

For more information about the Geospatial Studio see the docs page: [Geospatial Studio Docs](https://terrastackai.github.io/geospatial-studio)

For more information about the Geospatial Studio SDK and all the functions available through it, see the SDK docs page: [Geospatial Studio SDK Docs](https://terrastackai.github.io/geospatial-studio-toolkit)

### Get the training data
To train the AI model, we will need some training data which contains the input data and the labels (aka ground truth burn scar extent).  To train our model we will use the following dataset: https://huggingface.co/datasets/ibm-nasa-geospatial/hls_burn_scars

We can download it here: https://s3.us-east.cloud-object-storage.appdomain.cloud/geospatial-studio-example-data/burn-scar-training-data.zip

Download and unzip the above archive and if you wish you can explore the data with QGIS (or any similar tool).

*NB: If you already have the data in online you can skip this step.*


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install boto3


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
# Import the required packages
import json
import rasterio
import matplotlib.pyplot as plt

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from geostudio import Client
from geostudio import gswidgets

## Connecting to the platform
First, we set up the connection to the platform backend.  To do this we need the base url for the studio UI and an API key.

To get an API Key:
1. Go to the Geospatial Studio UI page and navigate to the Manage your API keys link. UI Link: [https://localhost:4180/](https:localhost:4180/)
2.  This should pop-up a window where you can generate, access and delete your api keys. NB: every user is limited to a maximum of two activate api keys at any one time.

Store the API key and geostudio ui base url in a credentials file locally, for example in /User/bob/.geostudio_config_file. You can do this by:

```bash
echo "GEOSTUDIO_API_KEY=<paste_api_key_here>" > .geostudio_config_file
echo "BASE_GATEWAY_API_URL=https://localhost:4181" >> .geostudio_config_file
```

Copy and paste the file path to this credentials file in call below.

In [16]:
#############################################################
# Initialize Geostudio client using a geostudio config file
#############################################################
gfm_client = Client(geostudio_config_file="../.geostudio_config_file_local")

Using api key and base urls from geostudio config file
Using api key and base urls from geostudio config file
Using api key and base urls from geostudio config file


## List and explore existing datasets in the platform

In [17]:
gfm_client.list_datasets(output="df")

,id,active,created_by,created_at,updated_at,dataset_name,description,dataset_url,label_suffix,purpose,data_sources,label_categories,size,status,error,logs
0,geodata-l7erwqeayrcpc7pksfmqqa,True,test@example.com,2026-01-28T08:59:22.083102Z,2026-01-28T09:53:44.335869Z,Burn Scars SDK demo,Burn Scars SDK data,https://s3.us-east.cloud-object-storage.appdom...,.mask.tif,Segmentation,"[{'bands': [{'index': '0', 'band_name': 'Blue'...","[{'id': '-1', 'name': 'Igone', 'color': '#0000...",0MB,Failed,New error - Problem downloading dataset: <urlo...,dflogs/2026-01-28/geodata-l7erwqeayrcpc7pksfmq...
1,geodata-ijbizkeqp47qopyb5vxad7,True,test@example.com,2026-01-23T12:16:55.798278Z,2026-01-23T12:52:39.672466Z,sen1floods11,The Sen1Floods11 dataset is a large-scale benc...,https://geospatial-studio-example-data.s3.us-e...,_LabelHand.tif,Segmentation,"[{'bands': [{'index': '0', 'band_name': 'VV', ...",[],1.8 GB,Succeeded,N/A,None


In [ ]:
# Copy the dataset_id of the dataset you want to explore further and replace with the dataset id below

gfm_client.get_dataset("geodata-ijbizkeqp47qopyb5vxad7", output="json")

## Data onboarding

In order to onboard your dataset to the Geospatial Studio, you need to have a direct download URL pointing to a zip file of the dataset. We will be using [this dataset](https://s3.us-east.cloud-object-storage.appdomain.cloud/geospatial-studio-example-data/burn-scar-training-data.zip) as an example to go through this notebook.

If you have the dataset locally, you can use Box, OneDrive or any other cloud storage you are used to.

Optionally, you can upload your data to a temporary location in the cloud (with in Studio object storage) and get a url which can be used to pass to the onboarding process. (*NB: the same upload function can be useful for pushing files for inferecnce or to processing pipelines.*)

The dataset needs to packaged as a zip file.

*Optional:* zip data files for upload:

`zip -j burn-scars-upload.zip /Downloads/burn-scars-upload/*`
```

In [ ]:
# # (Optional) If you wish to upload the data archive through the studio, you can use this function. Copy the path to your zipped dataset below.
# uploaded_links = gfm_client.upload_file('../../../geobench-datasets/burn-scar-training-data.zip')
# uploaded_links

##### Onboard the dataset to the dataset factory

Now we provide information about the dataset, including name, description, data and label file suffixes, dataset purpose, data sources, etc. Below is an example payload that defines most of the values you will need to onboard a dataset to the Studio. For more information on what you can provide during the onboarding process, check out the SDK Documentation

The Geospatial Studio allows users to onboard either multi-modal data or uni-modal data. For the multi-modal data, users shall provide, as a list, a different data source for each input modality of the dataset. 


Below are some example data connectors, collections and modality_tags to be provided in the dataset need to be correctly matched. See table below.  (The modality tags relate to the modalities in the Terramind model)

| Collections | Modality tag | Connector |
| :--- | :---: | ---: |
| s2_l1c | S2L1C | sentinelhub |
| dem | DEM | sentinelhub |
| s1_grd | S1GRD | sentinelhub |
| hls_l30 | HLS_L30 | sentinelhub |
| hls_s30 | HLS_S30 | sentinelhub |
| s2_l2a | S2L2A | sentinelhub |



In [10]:
# Edit the details in the dict and dataset_url below to suit your dataset

dataset_url = 'https://s3.us-east.cloud-object-storage.appdomain.cloud/geospatial-studio-example-data/burn-scar-training-data.zip'

dataset_dict = {
    "purpose": "Segmentation",
    "dataset_url": dataset_url,
    "label_suffix": ".mask.tif",
    "dataset_name": "Burn Scars SDK demo",
    "description": "Burn Scars SDK data",
    "data_sources": [
        {
            "bands": [
                {"index": "0", "band_name": "Blue", "scaling_factor": 0.0001, "RGB_band": "B"},
                {"index": "1", "band_name": "Green", "scaling_factor": 0.0001, "RGB_band": "G"},
                {"index": "2", "band_name": "Red", "scaling_factor": 0.0001, "RGB_band": "R"},
                {"index": "3", "band_name": "NIR_Narrow", "scaling_factor": 0.0001},
                {"index": "4", "band_name": "SWIR1", "scaling_factor": 0.0001},
                {"index": "5", "band_name": "SWIR2", "scaling_factor": 0.0001}
            ],
            "connector": "sentinelhub",
            "collection": "hls_l30",
            "file_suffix": "_merged.tif",
            "modality_tag": "HLS_L30"
        }
    ],
    "label_categories": [
        {"id": "-1", "name": "Igone", "color": "#000000", "opacity": "0", "weight": None},
        {"id": "0", "name": "NoData", "color": "#000000", "opacity": "0", "weight": None},
        {"id": "1", "name": "BurnScar", "color": "#ea7171", "opacity": 1, "weight": None}
    ],
    "version": "v2"
}

In [11]:
# start onboarding process

onboard_response = gfm_client.onboard_dataset(dataset_dict)
display(json.dumps(onboard_response, indent=2))

'{\n  "Dataset": "submitting - adding dataset and labels",\n  "dataset_id": "geodata-l7erwqeayrcpc7pksfmqqa",\n  "dataset_url": "https://s3.us-east.cloud-object-storage.appdomain.cloud/geospatial-studio-example-data/burn-scar-training-data.zip"\n}'

In [12]:
dataset_id = onboard_response["dataset_id"]
dataset_id

'geodata-l7erwqeayrcpc7pksfmqqa'

In [ ]:
# Poll onboarding status
gfm_client.poll_onboard_dataset_until_finished(onboard_response["dataset_id"])

## Fine-tuning submission

Once the data is onboarded, you are ready to setup your tuning task.  In order to run a fine-tuning task, you need to select the following items:
* **tuning task type** - what type of learning task are you attempting?  segmentation, regression etc
* **fine-tuning dataset** - what dataset will you use to train the model for your particular application?
* **base foundation model** - which geospatial foundation model will you use as the starting point for your tuning task?

Below we walk you through how to use the Geospatial Studio SDK to see what options are available in the platform for each of these, then once you have made your selection, how we configure our task and submit it.

### Tuning task
The tuning task tells the model what type of task it is (segmentation, regression etc), and exposes a range of optional hyperparameters which the user can set.  These all have reasonable defaults, but it gives uses the possibility to configure the model training how they wish.  Below, we will check what task templates are available to us, and then update some parameters.

Advanced users can create and upload new task templates to the platform, and instructions are found in the relevant notebook and documentation.  The templates are for Terratorch (the backend tuning library), and more details of Terratroch and configuration options can be found here: [https://terrastackai.github.io/terratorch/](https://terrastackai.github.io/terratorch/
)


In [8]:
tasks = gfm_client.list_tune_templates(output="df")
display(tasks[['name','description', 'id','created_by','updated_at']])

,name,description,id,created_by,updated_at
0,test-hpo-generated-template,Fine-tuned TerraTorch model for fire scar dete...,118eda79-913a-447c-8912-a51a1b67afd3,Beldine.Moturi@ibm.com,2025-10-30T11:10:04.444875Z
1,fire-scars-hpo-tune-generated-template,Fine-tuned TerraTorch model for fire scar dete...,4f798f65-3efd-40cc-b42d-33ccf08b932d,Beldine.Moturi@ibm.com,2025-10-28T10:12:00.617673Z
2,fire-scars-hpo-tune-003-generated-template,Fine-tuned TerraTorch model for fire scar dete...,ac9f499a-6f93-40d4-bf7d-00b1ed792ab5,Beldine.Moturi@ibm.com,2025-10-27T12:42:33.247356Z
3,user-new-task,user new task,8c868a2f-9fcb-487f-b117-406f0bbd49f8,Beldine.Moturi@ibm.com,2025-09-30T06:36:57.052712Z
4,timm_convnext : Segmentation,Segmentation of the convnext backbone models,2730a90e-ec83-4ad8-8eb8-98a633fa546a,system@ibm.com,2025-07-17T13:37:04.850000Z
5,timm_resnet : Segmentation,Segmentation of the resnet backbone models,a3189849-2db6-473b-b553-d56044a40e90,system@ibm.com,2025-07-17T13:23:53.544000Z
6,clay_v1 : Segmentation,Segmentation of the clay backbone models,fb490fc6-bd06-4e17-98cd-d8a8957bfe52,system@ibm.com,2025-09-08T10:07:41.909000Z
7,terramind Segmentation,Terramind multimodal task for Segmantation,48c878d8-3b05-4ca5-bd89-89400c8790eb,system@ibm.com,2025-06-27T15:43:33.944000Z
8,Segmentation,Generic template v1 and v2 models: Segmentation,e4791b2c-bb17-4a5e-9f05-1be5411a4fa6,system@ibm.com,2025-06-05T14:31:12.278000Z
9,Regression,Generic template for v1 & v2 models: Regression,d1137e61-58dc-4c56-b9db-25474e0944ad,system@ibm.com,2025-06-05T12:45:59.878000Z


In [9]:
# Choose a task from the options above.  Copy and paste the id into the variable, tid, below.
task_id = 'e4791b2c-bb17-4a5e-9f05-1be5411a4fa6'

In [10]:
# Now we can view the full meta-data and details of the selected task
task_meta = gfm_client.get_task(task_id, output="df")
task_meta

,id,active,created_by,created_at,updated_at,name,description,purpose,model_params.$uri,model_params.type,model_params.title,model_params.$schema,model_params.properties,model_params.description,extra_info.runtime_image,extra_info.model_category,extra_info.model_framework
0,e4791b2c-bb17-4a5e-9f05-1be5411a4fa6,True,system@ibm.com,2025-03-23T19:43:07.799000Z,2025-06-05T14:31:12.278000Z,Segmentation,Generic template v1 and v2 models: Segmentation,Segmentation,https://ibm.com/watsonx.ai.geospatial.finetune...,object,Finetune,https://json-schema.org/draft/2020-12/schema,"{'data': {'type': 'object', 'default': {'batch...",A request sent to the finetuning service to st...,us.icr.io/gfmaas/geostudio-ft-deploy:feat-upda...,prithvi,terratorch-v2


If you are happy with your choice, you can decide which (if any) hyperparameters you want to set (otherwise defaults will be used).

Here we can see the available parameters and their associated defaults.  To update a parameter you can just set values in the dictionary (as shown below for `max_epochs`).


In [11]:
task_params = gfm_client.get_task_param_defaults(task_id)
task_params

{'data': {'batch_size': 4, 'constant_multiply': 1, 'workers_per_gpu': 2},
 'model': {'decode_head': {'channels': 256,
   'num_convs': 4,
   'decoder': 'UNetDecoder',
   'loss_decode': {'type': 'CrossEntropyLoss', 'avg_non_ignore': True}},
  'frozen_backbone': False,
  'tiled_inference_parameters': {'h_crop': 224,
   'h_stride': 196,
   'w_crop': 224,
   'w_stride': 196,
   'average_patches': False}},
 'runner': {'max_epochs': 10,
  'early_stopping_patience': 20,
  'early_stopping_monitor': 'val/loss'},
 'lr_config': {'policy': 'Fixed'},
 'optimizer': {'lr': 6e-05, 'type': 'Adam'},
 'evaluation': {'interval': 1}}

In [ ]:
task_params['runner']['max_epochs'] = '2'
task_params['optimizer']['type'] = 'AdamW'
task_params['data']['batch_size'] = 4


### Base foundation model
The base model is the foundation model (encoder) which has been pre-trained and has the basic understanding of the data.  More information can currently be found on the different models in the documentation.


In [12]:
base = gfm_client.list_base_models(output='df')
display(base[['name','description','id','updated_at']])

,name,description,id,updated_at
0,sandbox-base-model,base model,d3428db7-000c-4b4d-b63f-9b4a534bfe3b,2025-10-27T12:42:33.674888Z
1,clay_v1_base,Clay is a foundational model of Earth. It uses...,ad729683-de1d-42f1-8279-cdb22cd3e67d,2025-07-14T09:22:17.369000Z
2,timm_resnet152,timm_resnet152,416600f2-4ad8-4e2d-88fe-8479040a4144,2025-07-14T09:19:07.168000Z
3,timm_resnet101,timm_resnet101,f9a1db55-5515-4a49-a545-8ede3b6e9d29,2025-07-14T09:18:54.928000Z
4,timm_resnet50,timm_resnet50,10807602-23d2-4b09-ad02-5a7185af32c1,2025-07-14T09:18:41.082000Z
5,timm_resnet18,timm_resnet18,bfa2bad0-917f-4aad-9075-ad8567c44b89,2025-07-14T09:18:28.036000Z
6,timm_resnet34,timm_resnet34,98fb087f-c30d-41d1-973f-46da372ab6ac,2025-07-14T09:17:48.267000Z
7,timm_convnext_xlarge.fb_in22k,timm_convnext_xlarge.fb_in22k,0f7515e6-f8e2-4b86-9aef-f226e7733ea0,2025-07-14T09:17:20.086000Z
8,timm_convnext_large.fb_in22k,timm_convnext_large.fb_in22k,d8490489-0afb-4e67-b35f-d97317a73555,2025-07-14T09:16:46.491000Z
9,Prithvi_EO_V2_600M_TL,Geospatial pre-traineed foundation model Prith...,4cc00692-d454-46e0-baa7-6a28bf6d5120,2025-05-20T13:23:17.724000Z


In [13]:
# copy and paste the id of the base model you wish to use

base_model_id = 'ba825321-426b-4c6a-9a4e-4a8fd2653ae6'

### Submitting the tune
Now we pull these choices together into a payload which we then submit to the platform.  This will then deploy the job in the backend and we will see below how we can monitor it.  First, we populate the payload so we can check it, then we simply submit. 

In [14]:
# create the tune payload

# dataset_id = onboard_response["dataset_id"] # the dataset_id of the dataset you onboarded above
dataset_id = "geodata-9agmakllxs7krdtvn8afvd"

tune_payload = {
  "name": "burn-scars-demo",
  "description": "Segmentation",
  "dataset_id": dataset_id,
  "base_model_id": base_model_id,
  "tune_template_id": task_id,
  # "model_parameters": task_params # uncomment this line if you customised task_params in the cells above otherwise, defaults will be used
}

print(json.dumps(tune_payload, indent=2))

{
  "name": "burn-scars-demo",
  "description": "Segmentation",
  "dataset_id": "geodata-9agmakllxs7krdtvn8afvd",
  "base_model_id": "ba825321-426b-4c6a-9a4e-4a8fd2653ae6",
  "tune_template_id": "e4791b2c-bb17-4a5e-9f05-1be5411a4fa6"
}


In [15]:
submitted = gfm_client.submit_tune(
        data = tune_payload,
        output = 'json'
)

print(submitted)

{'tune_id': 'geotune-es8gswvsx5wfgqcbtngvpq', 'mcad_id': 'kjob-geotune-es8gswvsx5wfgqcbtngvpq', 'status': 'In_progress', 'message': None}


## Monitoring training
Once the tune has been submitted you can check its status and monitor tuning progress through the SDK.  You can also access the training metrics and images in MLflow.  The `get_tune` function will give you the meta-data of the tune, including the status.


In [ ]:
# Poll fine tuning status
gfm_client.poll_finetuning_until_finished(tune_id=submitted["tune_id"])

In [ ]:
tune_id = submitted["tune_id"]

tune_info = gfm_client.get_tune(tune_id, output='df')
tune_info

Once it has started training, you will also be able to access the training metrics.  The `get_tune_metrics_df` function returns a dataframe containing the up-to-date training metrics, which you are free to explore and analyse.  In addition to that, you can simply plot the training and validation loss and multi-class accuracy using the `plot_tune_metrics` function.

In [ ]:
gfm_client.get_tune_metrics_df(tune_id)

In [ ]:
gswidgets.plot_tune_metrics(client=gfm_client, tune_id=tune_id)

Once your model is finished training and you are happy with the metrics (and images in MLflow), you can run some inference in test mode through the inference service.

## Testing your model

To do a test deployment and inference with the model, we need to register the model with the inference service.  To do this you need to select a model style (describing the visulisation style of the model output), and define the data required to feed the model (in the example here it is using Sentinel Hub).  For the data specification, you need to define the data collection and bands from sentinelhub (using the collection and band names for SH).  In addition, if the data to be fed in is returned from SH with a scale factor that needs to be added here too.  Data collection data for HLS are found here: https://docs.sentinel-hub.com/api/latest/data/hls/

**Example test locations**

|  Location            |  Date    | Bounding box      | 
| :---------------------: | :--------: | :-----------------: |
|  Park Fire, CA, USA (Cohasset, CA) | 2024-08-12 | [-121.837006, 39.826468, -121.641312, 40.038655] | 
|  Rhodes, Greece  | 2023-08-01 | [27.91, 35.99, 28.10, 36.25] |
|  Rafina, Greece  | 2018-08-04  | [23.92, 38.00, 24.03, 38.08] |
|  Bandipura State Forest, Karnataka, India  | 2019-02-26 | [76.503245, 11.631803, 76.690118, 11.762633]  |
|  Amur Oblast fires, Russia | 2018-05-29 | [127.589722, 54.055357, 128.960266, 54.701614]  |
|    |   |   |

#### Try out the model for inference
Once your model has finished tuning, if you want to run inference as a test you can do by passing either a location (bbox) or a url to a pre-prepared files.  The steps to test the model are:
1. Define the inference payload
2. Try out the tune temporarily

In [13]:
gfm_client.list_models(output="df")

,display_name,description,model_url,pipeline_steps,geoserver_push,model_input_data_spec,sharable,latest,version,internal_name,...,active,created_by,created_at,updated_at,status,model_onboarding_config.fine_tuned_model_id,model_onboarding_config.model_configs_url,model_onboarding_config.model_checkpoint_url,postprocessing_options.cloud_masking,postprocessing_options.ocean_masking
0,add-layer-sandbox-model,Early-access test model made available for dem...,None,"[{'status': 'READY', 'process_id': 'url-connec...",[],"[{'bands': [], 'connector': 'sentinelhub', 'co...",False,True,1.0,add-layer-sandbox-model-v1-25e2e6c4,...,True,test@example.com,2026-01-23T12:51:44.955591Z,2026-01-23T12:51:44.964507Z,PENDING,,,,NaN,NaN
1,geofm-sandbox-models,Early-access test model made available for dem...,None,"[{'status': 'READY', 'process_id': 'terrakit-d...","[{'workspace': 'geofm', 'layer_name': 'input_r...",[],True,True,1.0,geofm-sandbox-models-v1-c8850490,...,True,test@example.com,2026-01-23T12:51:44.733286Z,2026-01-23T12:51:44.803184Z,PENDING,,,,True,True
2,geofm-sandbox-models,Early-access test model made available for dem...,None,"[{'status': 'READY', 'process_id': 'terrakit-d...","[{'workspace': 'geofm', 'layer_name': 'input_r...",[],True,True,1.0,geofm-sandbox-models-v1-fd202fc0,...,True,Studio.Dev@dev.com,2026-01-23T12:00:27.259261Z,2026-01-23T12:00:27.282512Z,PENDING,,,,True,True


In [14]:
tune_id = "geotune-imupppagecf6rk6irpkirc"

In [27]:
tune_id=tune_id

payload={
"model_display_name": "geofm-sandbox-models",
"location": "Red Bluff, California, United States",
"description": "Park Fire Aug 2024",
"spatial_domain": {
  "bbox": [],
  "urls": [
    "https://geospatial-studio-example-data.s3.us-east.cloud-object-storage.appdomain.cloud/examples-for-inference/park_fire_scaled.tif"
  ],
  "tiles": [],
  "polygons": []
},
"temporal_domain": [
  "2024-08-12"
],
"pipeline_steps": [
  {
    "status": "READY",
    "process_id": "url-connector",
    "step_number": 0
  },
  {
    "status": "WAITING",
    "process_id": "terratorch-inference",
    "step_number": 1
  },
  {
    "status": "WAITING",
    "process_id": "postprocess-generic",
    "step_number": 2
  },
  {
    "status": "WAITING",
    "process_id": "push-to-geoserver",
    "step_number": 3
  }
],
"post_processing": {
  "cloud_masking": "False",
  "ocean_masking": "False",
  "snow_ice_masking": None,
  "permanent_water_masking": "False"
},
"model_input_data_spec": [
  {
    "bands": [
      {
        "index": "0",
        "RGB_band": "B",
        "band_name": "Blue",
        "scaling_factor": "0.0001"
      },
      {
        "index": "1",
        "RGB_band": "G",
        "band_name": "Green",
        "scaling_factor": "0.0001"
      },
      {
        "index": "2",
        "RGB_band": "R",
        "band_name": "Red",
        "scaling_factor": "0.0001"
      },
      {
        "index": "3",
        "band_name": "NIR_Narrow",
        "scaling_factor": "0.0001"
      },
      {
        "index": "4",
        "band_name": "SWIR1",
        "scaling_factor": "0.0001"
      },
      {
        "index": "5",
        "band_name": "SWIR2",
        "scaling_factor": "0.0001"
      }
    ],
    "connector": "sentinelhub",
    "collection": "hls_l30",
    "file_suffix": "_merged.tif",
    "modality_tag": "HLS_L30"
  }
],
"geoserver_push": [
  {
    "z_index": 0,
    "workspace": "geofm",
    "layer_name": "input_rgb",
    "file_suffix": "",
    "display_name": "Input image (RGB)",
    "filepath_key": "model_input_original_image_rgb",
    "geoserver_style": {
      "rgb": [
        {
          "label": "RedChannel",
          "channel": 1,
          "maxValue": 255,
          "minValue": 0
        },
        {
          "label": "GreenChannel",
          "channel": 2,
          "maxValue": 255,
          "minValue": 0
        },
        {
          "label": "BlueChannel",
          "channel": 3,
          "maxValue": 255,
          "minValue": 0
        }
      ]
    },
    "visible_by_default": "True"
  },
  {
    "z_index": 1,
    "workspace": "geofm",
    "layer_name": "pred",
    "file_suffix": "",
    "display_name": "Model prediction",
    "filepath_key": "model_output_image",
    "geoserver_style": {
      "segmentation": [
        {
          "color": "#000000",
          "label": "ignore",
          "opacity": 0,
          "quantity": "-1"
        },
        {
          "color": "#000000",
          "label": "no-data",
          "opacity": 0,
          "quantity": "0"
        },
        {
          "color": "#ab4f4f",
          "label": "fire-scar",
          "opacity": 1,
          "quantity": "1"
        }
      ]
    },
    "visible_by_default": "True"
  }
 ]
}



Once you have defined your inference payload, you can now run it with a test inference.  As with the main inference service, this is done by either supplying a bounding box (`bbox`), time range (`start_date`, `end_date`) and the `model_id`.  You can then monitor it and visualise the outputs either through the SDK, or in the UI.

In [28]:
# Now submit the test inference request
inference_response = gfm_client.try_out_tune(tune_id=tune_id, data=payload)
inference_response

{'spatial_domain': {'bbox': [],
  'polygons': [],
  'tiles': [],
  'urls': ['https://geospatial-studio-example-data.s3.us-east.cloud-object-storage.appdomain.cloud/examples-for-inference/park_fire_scaled.tif']},
 'temporal_domain': ['2024-08-12'],
 'fine_tuning_id': 'geotune-imupppagecf6rk6irpkirc',
 'maxcc': 100,
 'model_display_name': 'geofm-sandbox-models',
 'description': 'Park Fire Aug 2024',
 'location': 'Red Bluff, California, United States',
 'geoserver_layers': None,
 'demo': None,
 'model_id': '5080b078-d5b0-40f9-a4bf-ea887ebd736a',
 'inference_output': None,
 'id': '1bec6f6a-d965-47a3-845e-ff6e30165570',
 'active': True,
 'created_by': 'test@example.com',
 'created_at': '2026-01-28T09:40:24.258036Z',
 'updated_at': '2026-01-28T09:40:24.350538Z',
 'status': 'PENDING',
 'tasks_count_total': 1,
 'tasks_count_success': 0,
 'tasks_count_failed': 0,
 'tasks_count_stopped': 0,
 'tasks_count_waiting': 1}

In [6]:
gfm_client = Client(geostudio_config_file="../.geostudio_config_file_prod")

Using api key and base urls from geostudio config file
Using api key and base urls from geostudio config file
Using api key and base urls from geostudio config file


In [7]:
gfm_client.list_models(output="df")

,display_name,description,model_url,pipeline_steps,geoserver_push,model_input_data_spec,sharable,latest,version,internal_name,...,active,created_by,created_at,updated_at,status,postprocessing_options.cloud_masking,postprocessing_options.ocean_masking,model_onboarding_config.fine_tuned_model_id,model_onboarding_config.model_configs_url,model_onboarding_config.model_checkpoint_url
0,prithvi-eo-flood,senflood11_swin with terratorch,https://amo-prithvi-eo-flood-geospatial-be.app...,"[{'status': 'READY', 'process_id': 'terrakit-d...","[{'workspace': 'geofm', 'layer_name': 'input_r...","[{'bands': [{'index': 0, 'band_name': 'B02', '...",True,True,2.0,prithvi-eo-flood,...,True,system@ibm.com,2025-08-25T10:13:53.348567Z,2025-08-25T10:13:53.465461Z,COMPLETED,True,True,,,
1,add-layer-sandbox-models,Early-access test model made available for dem...,None,"[{'status': 'READY', 'process_id': 'url-connec...",[],"[{'bands': [{'index': 0, 'band_name': 'B02', '...",True,True,1.0,add-layer-sandbox-model-v1-a1557sep,...,True,Fred.Otieno@ibm.com,2025-08-15T07:12:07.403000Z,2025-09-05T14:47:53.826000Z,COMPLETED,NaN,NaN,,,
2,add-layer-sandbox-model,Early-access test model made available for dem...,None,"[{'status': 'READY', 'process_id': 'url-connec...",[],"[{'bands': [{'index': 0, 'band_name': 'B02', '...",True,True,1.0,add-layer-sandbox-model-v1-a1558sep,...,True,Fred.Otieno@ibm.com,2025-08-15T07:12:07.403000Z,2025-09-05T14:47:53.826000Z,COMPLETED,NaN,NaN,,,
3,geofm-sandbox-models,Early-access test model made available for dem...,None,"[{'status': 'READY', 'process_id': 'terrakit-d...","[{'workspace': 'geofm', 'layer_name': 'input_r...",[],True,True,1.0,geofm-sandbox-models-v1-4e75e950,...,True,system@ibm.com,2025-06-11T09:55:57.456000Z,2025-06-11T09:55:57.485000Z,COMPLETED,True,True,None,None,None


In [ ]:
# define the inference payload

bbox = [-121.837006,39.826468,-121.641312,40.038655]

request_payload = {
	"description": "Park Fire 2024 SDK",
	"location": "Red Bluff, California, United States",
	"spatial_domain": {
			"bbox": [bbox],
			"polygons": [],
			"tiles": [],
			"urls": []
	},
	"temporal_domain": [
			"2024-08-12_2024-08-13"
	]
}

## Monitoring your inference task

Once submitted you can check on progress using the following function which will return all the metadata about the inference task, including the status.  You can optionally use the `poll_until_finished` to watch the status until it completes.  For a test inference it can take 5-10 minutes, depending on the size of the data query, the size of the model etc.

In [ ]:
gfm_client.get_inference(inference_response['id'], output="df")

In [ ]:
# Poll inference status
gfm_client.poll_inference_until_finished(inference_id=inference_response['id'], poll_frequency=10)

## Accessing inference outputs
Once an inference run is completed, the inputs and outputs of each task within an inference are packaged up into a zip file which is uploaded to a url you can use to download the files.

To access the inference task files:
1. Get the inference tasks list
2. Identify the specific inference task you want to view
3. Download task output files

In [ ]:
# Get the inference tasks list
inf_tasks_res = gfm_client.get_inference_tasks(inference_response["id"])
inf_tasks_res

In [ ]:
df = gfm_client.inference_task_status_df(inference_response["id"])


display(df.style.map(gswidgets.color_inference_tasks_by_status))

In [ ]:
gswidgets.view_inference_process_timeline(gfm_client, inference_id = inference_response["id"])

Next, Identify the task you want to view from the response above, ensure status of the task is FINISHED and set `selected_task` variable below to the task number at the end of the task id string. For example, if `task_id` is "6d1149fa-302d-4612-82dd-5879fc06081d-task_0", selected_task woul be 0

In [ ]:
# Select a task to view

selected_task = 0 
selected_task_id = f"{inference_response["id"]}-task_{selected_task}"

In [ ]:
# Download task output files

gswidgets.fileDownloaderTasks(client=gfm_client, task_id=selected_task_id)

## Visualizing the output of the inference runs

You can check out the results visually in the Studio UI, or with the quick widget below.  You can alternatively use the SDK to download selected files for further analysis [see documentation](https://github.ibm.com/GeospatialStudio/GeospatialStudioExamples/blob/main/Inference/GeospatialStudio-Example01-Inference.ipynb).

We have several options for visualising the data:
* we can load the data with a package like rasterio and plot the images, and/or access the values.
* we could use the widget from the SDK to visualise the chosen files for a inference run. (shown below)
* view the data in the Geospatial Studio Inference lab UI.
* load the files in an external software, such as QGIS.

#### Load the data with a package rasterio and plot the images, and/or access the values.

In [ ]:
# Paste the name (+path) to one of the files you downloaded and select the band you want to load+plot
filename = '9a0588de-8858-4c31-8c6d-91b9c74333d1-task_0_HLS_L30_2024-08-12__merged.tif.tif'
band_number = 1

# open the file and read the band and metadata with rasterio
with rasterio.open(filename) as fp:
    data = fp.read(band_number)
    bounds = fp.bounds


print("Image dimensions: " + str(data.shape))

plt.imshow(data, extent=[bounds.left, bounds.right, bounds.bottom, bounds.top])
plt.xlabel('Longitude'); plt.xlabel('Latitude')

#### Visualize through the SDK widgets

In [ ]:
# Visualize output files with the SDK
gswidgets.inferenceTaskViewer(client=gfm_client, task_id=selected_task_id)